# Сравнительный анализ моделей аспектного анализа тональности русских отзывов

## Цель исследования
Провести сравнительный анализ готовых моделей для анализа тональности русских отзывов с HuggingFace.

## Модели для сравнения:
1. **blanchefort/rubert-base-cased-sentiment-rurewiews** - RuBERT на отзывах RuReviews
2. **sismetanin/rubert-ru-sentiment-rureviews** - RuBERT на отзывах e-commerce
3. **seara/rubert-tiny2-russian-sentiment** - Легковесная версия RuBERT
4. **sismetanin/xlm_roberta_base-ru-sentiment-rureviews** - XLM-RoBERTa для русского языка

## Метрики оценки:
- Accuracy
- F1-score (macro/weighted)
- Precision и Recall
- Время инференса
- Размер модели

In [1]:
# Установка необходимых библиотек
# !pip install transformers torch datasets pandas scikit-learn matplotlib seaborn tqdm

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from datasets import load_dataset
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import time
import warnings
from pathlib import Path
import ssl
import urllib.request
warnings.filterwarnings('ignore')

# Отключение проверки SSL сертификатов (для корпоративных сетей)
ssl._create_default_https_context = ssl._create_unverified_context

# Для requests (используется HuggingFace)
import os
os.environ['CURL_CA_BUNDLE'] = ''
os.environ['REQUESTS_CA_BUNDLE'] = ''

# Настройка отображения
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Пути к файлам
RESEARCH_DIR = Path.cwd()
print(f"Рабочая директория: {RESEARCH_DIR}")

# Проверка доступности GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Используемое устройство: {device}')
print("\nПримечание: SSL проверка отключена для загрузки моделей")

## 1. Подготовка тестовых данных

Создадим набор тестовых русских отзывов с разными аспектами и тональностями.

In [ ]:
# Создаем тестовые отзывы с известной тональностью
test_reviews = [
    # Позитивные отзывы
    {"text": "Отличный отель! Номера чистые, персонал вежливый, завтрак превосходный.", "label": "POSITIVE"},
    {"text": "Прекрасный ресторан, вкусная еда, быстрое обслуживание. Обязательно вернусь!", "label": "POSITIVE"},
    {"text": "Товар пришел быстро, качество отличное, упаковка хорошая. Рекомендую!", "label": "POSITIVE"},
    {"text": "Замечательный ноутбук, быстрый, легкий, батарея держит долго.", "label": "POSITIVE"},
    {"text": "Очень довольна покупкой! Платье красивое, ткань приятная, сидит идеально.", "label": "POSITIVE"},
    
    # Негативные отзывы
    {"text": "Ужасный отель! Грязные номера, невежливый персонал, плохой завтрак.", "label": "NEGATIVE"},
    {"text": "Разочарован покупкой. Товар низкого качества, долгая доставка, плохая упаковка.", "label": "NEGATIVE"},
    {"text": "Не рекомендую этот ресторан. Еда невкусная, обслуживание медленное, цены завышены.", "label": "NEGATIVE"},
    {"text": "Ноутбук постоянно зависает, батарея быстро разряжается, очень тяжелый.", "label": "NEGATIVE"},
    {"text": "Платье пришло не того размера, ткань дешевая, швы кривые. Деньги на ветер.", "label": "NEGATIVE"},
    
    # Нейтральные отзывы
    {"text": "Обычный отель, ничего особенного. Номера стандартные, персонал нормальный.", "label": "NEUTRAL"},
    {"text": "Товар соответствует описанию. Доставка в срок.", "label": "NEUTRAL"},
    {"text": "Ресторан как ресторан, еда средняя, цены приемлемые.", "label": "NEUTRAL"},
    {"text": "Ноутбук подходит для базовых задач. Ничего выдающегося.", "label": "NEUTRAL"},
    
    # Смешанные (аспектные) отзывы
    {"text": "Отель хороший, но завтрак разочаровал. Номера чистые, но персонал грубый.", "label": "NEUTRAL"},
    {"text": "Еда в ресторане отличная, но ждали очень долго. Атмосфера приятная, но цены высокие.", "label": "NEUTRAL"},
    {"text": "Ноутбук мощный, но очень шумный. Экран яркий, но батарея слабая.", "label": "NEUTRAL"},
    {"text": "Качество товара хорошее, но доставка задержалась на неделю.", "label": "NEUTRAL"},
    {"text": "Платье красивое, но маломерит. Ткань качественная, но дорого.", "label": "NEUTRAL"},
    
    # Дополнительные позитивные
    {"text": "Превосходное качество обслуживания! Всё на высшем уровне.", "label": "POSITIVE"},
    {"text": "Лучший отель в городе! Всем рекомендую.", "label": "POSITIVE"},
    
    # Дополнительные негативные
    {"text": "Полное разочарование. Не советую никому.", "label": "NEGATIVE"},
    {"text": "Потраченные деньги жалко. Очень плохо.", "label": "NEGATIVE"},
]

# Преобразуем в DataFrame
df_test = pd.DataFrame(test_reviews)
print(f"Создано тестовых отзывов: {len(df_test)}")
print(f"\nРаспределение по классам:")
print(df_test['label'].value_counts())
df_test.head(10)

## 2. Загрузка моделей

Загружаем все модели для сравнения.

In [ ]:
# Список моделей для тестирования
model_names = [
    "blanchefort/rubert-base-cased-sentiment-rurewiews",
    "sismetanin/rubert-ru-sentiment-rureviews",
    "seara/rubert-tiny2-russian-sentiment",
    "sismetanin/xlm_roberta_base-ru-sentiment-rureviews",
]

# Директория для локального хранения моделей
MODELS_CACHE_DIR = RESEARCH_DIR / 'models_cache'
MODELS_CACHE_DIR.mkdir(exist_ok=True)

print(f"Модели будут сохраняться в: {MODELS_CACHE_DIR}\n")

# Словарь для хранения pipelines
models = {}

print("Загрузка моделей...\n")
for model_name in model_names:
    print(f"Загружается: {model_name}")
    
    # Короткое имя для локального хранения
    short_name = model_name.split('/')[-1]
    local_model_path = MODELS_CACHE_DIR / short_name
    
    try:
        # Проверяем, есть ли модель локально
        if local_model_path.exists() and (local_model_path / 'config.json').exists():
            print(f"  Найдена локальная копия, загружаем из {local_model_path}")
            pipe = pipeline(
                "sentiment-analysis",
                model=str(local_model_path),
                device=0 if device == 'cuda' else -1
            )
        else:
            print(f"  Скачивание с HuggingFace...")
            pipe = pipeline(
                "sentiment-analysis",
                model=model_name,
                device=0 if device == 'cuda' else -1
            )
            # Сохраняем модель локально
            print(f"  Сохранение в {local_model_path}")
            pipe.model.save_pretrained(local_model_path)
            pipe.tokenizer.save_pretrained(local_model_path)
        
        models[short_name] = pipe
        print(f"  Загружена успешно\n")
        
    except Exception as e:
        print(f"  Ошибка загрузки: {e}\n")

print(f"Всего загружено моделей: {len(models)}")
print(f"\nМодели сохранены в: {MODELS_CACHE_DIR}")
print("При следующем запуске будут загружены из локального кэша")

## 3. Получение информации о моделях

In [ ]:
# Информация о моделях
model_info = []

for short_name, pipe in models.items():
    model = pipe.model
    # Подсчет параметров
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    model_info.append({
        'Модель': short_name,
        'Всего параметров': f"{total_params:,}",
        'Обучаемых параметров': f"{trainable_params:,}",
        'Классы': pipe.model.config.id2label if hasattr(pipe.model.config, 'id2label') else 'N/A'
    })

df_info = pd.DataFrame(model_info)
print("Информация о загруженных моделях:\n")
df_info

## 4. Тестирование моделей

Прогоняем все отзывы через каждую модель и собираем результаты.

In [ ]:
# Функция для нормализации меток
def normalize_label(label):
    """Нормализует различные форматы меток к стандартному виду"""
    label = str(label).upper()
    
    if 'POSITIVE' in label or 'LABEL_2' in label or label == '2':
        return 'POSITIVE'
    elif 'NEGATIVE' in label or 'LABEL_0' in label or label == '0':
        return 'NEGATIVE'
    elif 'NEUTRAL' in label or 'LABEL_1' in label or label == '1':
        return 'NEUTRAL'
    else:
        return label

# Функция для тестирования модели
def test_model(pipe, texts, batch_size=8):
    """Тестирует модель на списке текстов"""
    predictions = []
    scores = []
    inference_times = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Обработка батчей"):
        batch = texts[i:i+batch_size]
        
        start_time = time.time()
        results = pipe(batch)
        batch_time = time.time() - start_time
        
        for result in results:
            predictions.append(normalize_label(result['label']))
            scores.append(result['score'])
            inference_times.append(batch_time / len(batch))
    
    return predictions, scores, inference_times

# Тестируем все модели
results = {}
texts = df_test['text'].tolist()
true_labels = df_test['label'].tolist()

print("Тестирование моделей на отзывах...\n")
for model_name, pipe in models.items():
    print(f"\nТестирование: {model_name}")
    predictions, scores, times = test_model(pipe, texts)
    
    results[model_name] = {
        'predictions': predictions,
        'scores': scores,
        'times': times,
        'avg_time': np.mean(times)
    }
    
    print(f"Средняя время инференса: {results[model_name]['avg_time']:.4f} сек/отзыв")

print("\nВсе модели протестированы!")

## 5. Оценка качества моделей

Вычисляем метрики для каждой модели.

In [ ]:
# Вычисляем метрики для каждой модели
metrics_data = []

for model_name, result in results.items():
    predictions = result['predictions']
    
    # Вычисляем метрики
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels, predictions, average='weighted', zero_division=0
    )
    
    # Macro метрики
    _, _, f1_macro, _ = precision_recall_fscore_support(
        true_labels, predictions, average='macro', zero_division=0
    )
    
    metrics_data.append({
        'Модель': model_name,
        'Accuracy': f"{accuracy:.4f}",
        'Precision (weighted)': f"{precision:.4f}",
        'Recall (weighted)': f"{recall:.4f}",
        'F1-score (weighted)': f"{f1:.4f}",
        'F1-score (macro)': f"{f1_macro:.4f}",
        'Avg Inference Time (s)': f"{result['avg_time']:.4f}"
    })

df_metrics = pd.DataFrame(metrics_data)
print("Метрики качества моделей:\n")
df_metrics

## 6. Детальные отчеты по классификации

In [ ]:
# Детальные отчеты для каждой модели
for model_name, result in results.items():
    print(f"\n{'='*80}")
    print(f"Модель: {model_name}")
    print(f"{'='*80}\n")
    
    predictions = result['predictions']
    print(classification_report(true_labels, predictions, zero_division=0))

## 7. Визуализация результатов

In [ ]:
# Матрицы ошибок для всех моделей
n_models = len(models)
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for idx, (model_name, result) in enumerate(results.items()):
    if idx < len(axes):
        cm = confusion_matrix(true_labels, result['predictions'], 
                             labels=['NEGATIVE', 'NEUTRAL', 'POSITIVE'])
        
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                   xticklabels=['NEGATIVE', 'NEUTRAL', 'POSITIVE'],
                   yticklabels=['NEGATIVE', 'NEUTRAL', 'POSITIVE'])
        axes[idx].set_title(f'Confusion Matrix: {model_name}', fontsize=10)
        axes[idx].set_ylabel('True Label')
        axes[idx].set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig(RESEARCH_DIR / 'confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()
print("Матрицы ошибок сохранены")

In [ ]:
# Сравнение метрик
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# График 1: Accuracy и F1-score
metrics_comparison = []
for model_name, result in results.items():
    predictions = result['predictions']
    acc = accuracy_score(true_labels, predictions)
    _, _, f1_weighted, _ = precision_recall_fscore_support(
        true_labels, predictions, average='weighted', zero_division=0
    )
    _, _, f1_macro, _ = precision_recall_fscore_support(
        true_labels, predictions, average='macro', zero_division=0
    )
    
    metrics_comparison.append({
        'Model': model_name,
        'Accuracy': acc,
        'F1 (weighted)': f1_weighted,
        'F1 (macro)': f1_macro
    })

df_comp = pd.DataFrame(metrics_comparison)
df_comp.set_index('Model')[['Accuracy', 'F1 (weighted)', 'F1 (macro)']].plot(
    kind='bar', ax=axes[0], rot=45
)
axes[0].set_title('Сравнение метрик качества', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Значение метрики')
axes[0].set_xlabel('')
axes[0].legend(loc='lower right')
axes[0].set_ylim([0, 1.0])
axes[0].grid(axis='y', alpha=0.3)

# График 2: Время инференса
times_data = [(name, result['avg_time']) for name, result in results.items()]
models_names, avg_times = zip(*times_data)
axes[1].bar(range(len(models_names)), avg_times, color='coral')
axes[1].set_xticks(range(len(models_names)))
axes[1].set_xticklabels(models_names, rotation=45, ha='right')
axes[1].set_title('Среднее время инференса', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Время (секунды)')
axes[1].set_xlabel('')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(RESEARCH_DIR / 'metrics_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("Сравнительные графики сохранены")

## 8. Анализ примеров предсказаний

In [ ]:
# Создаем таблицу с примерами предсказаний
examples_df = df_test.copy()

for model_name, result in results.items():
    examples_df[f'{model_name}_pred'] = result['predictions']
    examples_df[f'{model_name}_score'] = [f"{s:.3f}" for s in result['scores']]

print("Примеры предсказаний всех моделей:\n")
examples_df.head(15)

In [ ]:
# Анализ расхождений между моделями
print("Анализ расхождений между моделями:\n")

# Найдем отзывы где модели не согласны
model_names_list = list(results.keys())
disagreements = []

for idx, row in examples_df.iterrows():
    predictions = [row[f'{model}_pred'] for model in model_names_list]
    
    # Если не все предсказания одинаковые
    if len(set(predictions)) > 1:
        disagreements.append({
            'Текст': row['text'][:60] + '...',
            'Истинная метка': row['label'],
            **{f'{model}': row[f'{model}_pred'] for model in model_names_list}
        })

if disagreements:
    df_disagree = pd.DataFrame(disagreements)
    print(f"Найдено {len(df_disagree)} отзывов с расхождениями в предсказаниях:\n")
    display(df_disagree)
else:
    print("Все модели согласны во всех предсказаниях!")

## 9. Итоговые выводы и рекомендации

In [ ]:
# Находим лучшую модель по разным критериям
best_accuracy = max(metrics_comparison, key=lambda x: x['Accuracy'])
best_f1 = max(metrics_comparison, key=lambda x: x['F1 (weighted)'])
fastest = min(times_data, key=lambda x: x[1])

print("="*80)
print("ИТОГОВЫЕ РЕЗУЛЬТАТЫ СРАВНИТЕЛЬНОГО АНАЛИЗА")
print("="*80)
print(f"\nЛучшая accuracy: {best_accuracy['Model']} ({best_accuracy['Accuracy']:.4f})")
print(f"Лучший F1-score: {best_f1['Model']} ({best_f1['F1 (weighted)']:.4f})")
print(f"Самая быстрая: {fastest[0]} ({fastest[1]:.4f} сек/отзыв)")
print("\n" + "="*80)
print("\nРЕКОМЕНДАЦИИ:")
print("- Для максимальной точности: используйте модели на основе RuBERT-base")
print("- Для быстрого инференса: используйте RuBERT-tiny2")
print("- Для продакшн-систем: баланс между RuBERT и XLM-RoBERTa")
print("\n" + "="*80)

## 10. Сохранение результатов

In [ ]:
# Сохраняем результаты в CSV
df_metrics.to_csv(RESEARCH_DIR / 'model_metrics.csv', index=False)
examples_df.to_csv(RESEARCH_DIR / 'predictions.csv', index=False)

print("Результаты сохранены в файлы:")
print("  - model_metrics.csv")
print("  - predictions.csv")
print("  - confusion_matrices.png")
print("  - metrics_comparison.png")

## 11. Ожидаемые результаты и выводы

### Цель анализа

В результате исследования предполагается определить, какие модели демонстрируют наиболее высокую эффективность при анализе русскоязычных отзывов, а также выявить ограничения существующих решений.

### Ожидаемые результаты

#### 1. Сравнительная таблица результатов

По итогам тестирования ожидается получить следующие данные:

**Метрики качества классификации:**
- Accuracy (точность) - общая доля правильных предсказаний
- F1-score (weighted/macro) - гармоническое среднее precision и recall
- Precision - точность положительных предсказаний
- Recall - полнота обнаружения классов

**Метрики производительности:**
- Время инференса на один отзыв
- Размер модели (количество параметров)

#### 2. Выявленные паттерны

**Ожидается, что:**

- **RuBERT-base модели** покажут высокую точность благодаря предобучению на русскоязычных корпусах
- **RuBERT-tiny** будет быстрее, но может уступать в точности
- **XLM-RoBERTa** продемонстрирует баланс между качеством и универсальностью
- Модели могут испытывать трудности с **аспектными (смешанными) отзывами**, где присутствуют как позитивные, так и негативные оценки

#### 3. Ограничения текущих решений

**Возможные проблемы:**

1. **Аспектный анализ:** Стандартные модели sentiment analysis классифицируют текст целиком, не выделяя отдельные аспекты (качество, сервис, цена)

2. **Контекстная чувствительность:** Отзывы типа "Отель хороший, НО завтрак плохой" могут классифицироваться неоднозначно

3. **Доменная специфика:** Модели, обученные на одном типе отзывов (e-commerce), могут хуже работать на других доменах (отели, рестораны)

4. **Размер модели vs качество:** Компактные модели быстрее, но теряют в точности

### Аналитическое заключение

**По результатам исследования будет сформулировано:**

1. **Рейтинг моделей** по различным критериям:
   - Лучшая по точности
   - Оптимальная по соотношению скорость/качество
   - Рекомендуемая для production-систем

2. **Рекомендации по применению:**
   - Для каких задач какая модель подходит лучше
   - Когда стоит использовать легковесные модели
   - В каких случаях нужны более тяжелые архитектуры

3. **Направления улучшения:**
   - Дообучение (fine-tuning) на специфичных данных
   - Использование ансамблей моделей
   - Применение специализированных моделей для аспектного анализа

### Практическая значимость

Результаты исследования помогут:

- **Разработчикам** выбрать оптимальную модель для своих задач
- **Исследователям** понять текущее состояние sentiment analysis для русского языка
- **Бизнесу** оценить возможности автоматизации анализа отзывов

### Воспроизводимость

Все эксперименты выполняются на фиксированном наборе тестовых отзывов, что обеспечивает:
- Объективность сравнения
- Возможность воспроизведения результатов
- Прозрачность методологии оценки

---

**Итоговый вывод:** Сравнительный анализ готовых моделей позволит сформировать практические рекомендации по выбору инструментов для анализа тональности русскоязычных отзывов и определить направления дальнейших исследований в этой области.